# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [6]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, on='movieId', how='left')

def getClass(row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] <= 4:
        return 'средний рейтинг'
    elif row['rating'] <= 5 and row['rating'] >= 4.5:
        return 'высокий рейтинг'
    else:
        return 'undefined'

joined['class'] = joined.apply(getClass, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,средний рейтинг
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,средний рейтинг
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,средний рейтинг
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,низкий рейтинг
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,средний рейтинг


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [29]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
keywords = pd.read_csv('ml-latest-small/keywords.csv')

def getRegion(row):
    keywordList = row['keyword'].split(' ')
    for key in keywordList:
        result = searchRegion(key, geo_data)
        if result != False:
            return result
    return 'undefined'
    
def searchRegion(word, geo_data):
    for geo in geo_data:
        if word.lower() in geo_data[geo]:
            return geo
    return False

keywords['region'] = keywords.apply(getRegion, axis=1)

notUndefined = keywords[ keywords.region != 'undefined' ]
notUndefined.head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


# Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [40]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, on='movieId', how='left')

years = [*range(1950, 2010, 1)] 

def production_year(row, years):
    """
        - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    """
    for year in years:
        if row['title'].find(str(year)) != -1:
            return year
    return 1900
joined['year'] = joined.apply(production_year, years=years, axis=1)

joined[['year', 'rating']].groupby('year').mean().sort_values('rating', ascending=False).head()

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
